### This notebook assumes that your working directory is the Breton_corvée folder. Currently takes ogee from google drive instad of file.


# SECTION 1: Dataset overlap

In [ ]:
## install required packages
if (!require("pacman")) install.packages("pacman")

# # Needed to install sf on Mac, currently unused.
# if(Sys.info()["sysname"] == 'Darwin'){ #if on osx, install udunits - required to install units
#     system("brew install udunits")
# } 

pacman::p_load(raster, sp, rgeos, geosphere, rgdal, geojsonio, ggplot2, readr, tidyr, devtools, googledrive, 
               venneuler, rapportools, measurements)
# pacman::p_load(sf)
devtools::install_github("tidyverse/googlesheets4")
library(googlesheets4)

## Read map, cassini, and insee from csv files, ogee from google drive
### (Potential TODO: pull all files from google drive instead)

In [ ]:
## store the URL you have
url = 'https://docs.google.com/spreadsheets/d/19dbb5Os-nZ5SU7V0kP10y9k33xZEF7W1aE1eaUcRyXs'

## identify this folder on Drive
## may prompt you to authorize
ss = drive_get(as_id(url))

## may prompt you to authorize
ogee = as.data.frame(read_sheet(ss, sheet = "Ogée 1778"))
names(ogee)[which(names(ogee) == 'INSEE ID')] = 'INSEE.ID' #fix column name
ogee2 = ogee #make a backup for use in section 2

# ogee = read.csv("ogee.csv") #enable if you instead want ogee from csv, may break other code

cassini = read.csv("chef_lieux.csv")
names(cassini)[which(names(cassini) == 'insee_l')] = 'INSEE.ID' #fix column name

map = read.csv("map.csv")


insee = read.csv("insee.csv")


## Data cleaning

In [ ]:
#replace accented characters
rep = 'aeecaeiouueiuAEECAEIOUUEIU'
orig = 'àèéçâêîôûùëïüÀÈÉÇÂÊÎÔÛÙËÏÜ'

cassini[] <- lapply(cassini, chartr, old=orig, new=rep)
ogee[] <- lapply(ogee, chartr, old=orig, new=rep)
map[] <- lapply(map, chartr, old=orig, new=rep)
cassini[] <- lapply(cassini, gsub, pattern='Œ', replacement='OE')
cassini[] <- lapply(cassini, gsub, pattern='œ', replacement='oe')
ogee[] <- lapply(ogee, gsub, pattern='Œ', replacement='OE')
ogee[] <- lapply(ogee, gsub, pattern='œ', replacement='oe')
map[] <- lapply(map, gsub, pattern='Œ', replacement='OE')
map[] <- lapply(map, gsub, pattern='œ', replacement='oe')


In [ ]:
# Filter out rows by assignment, disabled for now

# assignment_vec <- c('Y') # add acceptable statuses here
# map <- map[map$assignment %in% assignment_vec, ]
# status_vec <- c('KM5', 'MC5', 'SL4', 'SL5', 'EM4', 'EM5', 'EK4', 'EK5') # add acceptable statuses here
# ogee <- ogee[ogee$Status %in% status_vec, ]

In [ ]:
# Drop NA's from INSEE, disabled for now
# insee <- insee %>% drop_na()

In [ ]:
# Add additional CLEAN_NAME column for comparison when INSEE ID fails or is not available
nom <- cassini$label
nom <- toupper(nom)
cassini$CLEAN_NAME <- nom

nom <- ogee$'Ogée 1778 entry'
nom <- toupper(nom)
ogee$CLEAN_NAME <- nom
ogee[which(is.na(ogee[,'CLEAN_NAME'])), 'CLEAN_NAME'] = ogee[which(is.na(ogee[,'CLEAN_NAME'])), 'NOM']

nom <- map$normalized.place.name
nom <- toupper(nom)
map$CLEAN_NAME <- nom

nom <- insee$NOM
nom <- toupper(nom)
insee$CLEAN_NAME <- nom

# Filter out unnecessary columns
insee_keeps <- c('long', 'lat', 'INSEE.ID', 'CLEAN_NAME')
insee <- insee[insee_keeps]
map_keeps <- c('INSEE.ID', 'CLEAN_NAME')
map <- map[map_keeps]
ogee_keeps <- c('INSEE.ID', 'CLEAN_NAME')
ogee <- ogee[ogee_keeps]
cassini_keeps = c('xlamb', 'ylamb', 'lon', 'lat', 'CLEAN_NAME', 'INSEE.ID')
cassini <- cassini[cassini_keeps]
names(cassini)[which(names(cassini) == 'lon')] = 'longcassini' #make column names unique
names(cassini)[which(names(cassini) == 'lat')] = 'latcassini' #make column names unique

#Add origin statuses
insee['INSEE_CHECK'] = TRUE
map['MAP_CHECK'] = TRUE
cassini['CASSINI_CHECK'] = TRUE
ogee['OGEE_CHECK'] = TRUE

### Limit datasets to Brittany based on Insee ID. Note: this will remove entries with missing or malformed ids,
### and should probably be treated only as a temporary solution

In [ ]:
#limit datasets to Brittany based on Insee ID. Note: this will remove entries with missing or malformed ids,
#and should probably be treated only as a temporary solution
cidx = c(which(startsWith(as.character(cassini[['INSEE.ID']]), "44")),
         which(startsWith(as.character(cassini[['INSEE.ID']]), "35")),
          which(startsWith(as.character(cassini[['INSEE.ID']]), "22")),
          which(startsWith(as.character(cassini[['INSEE.ID']]), "29")),
          which(startsWith(as.character(cassini[['INSEE.ID']]), "56")))

midx = c(which(startsWith(as.character(map[['INSEE.ID']]), "44")),
         which(startsWith(as.character(map[['INSEE.ID']]), "35")),
          which(startsWith(as.character(map[['INSEE.ID']]), "22")),
          which(startsWith(as.character(map[['INSEE.ID']]), "29")),
          which(startsWith(as.character(map[['INSEE.ID']]), "56")))

iidx = c(which(startsWith(as.character(insee[['INSEE.ID']]), "44")),
         which(startsWith(as.character(insee[['INSEE.ID']]), "35")),
          which(startsWith(as.character(insee[['INSEE.ID']]), "22")),
          which(startsWith(as.character(insee[['INSEE.ID']]), "29")),
          which(startsWith(as.character(insee[['INSEE.ID']]), "56")))

# oidx = c(which(startsWith(as.character(ogee[['INSEE.ID']]), "44")),
#          which(startsWith(as.character(ogee[['INSEE.ID']]), "35")),
#           which(startsWith(as.character(ogee[['INSEE.ID']]), "22")),
#           which(startsWith(as.character(ogee[['INSEE.ID']]), "29")),
#           which(startsWith(as.character(ogee[['INSEE.ID']]), "56")))

cassini = cassini[cidx,]
map = map[midx,]
insee = insee[iidx,]
# ogee = ogee[iidx,]

## Merge based on Insee ID and clean name (for when no ID is provided)

In [ ]:
outer_1 = merge(ogee, insee, by = c('INSEE.ID', 'CLEAN_NAME'), all = TRUE, suffixes = c('.ogee', '.insee'))
outer_2 = merge(outer_1, map, c('INSEE.ID', 'CLEAN_NAME'), all = TRUE, suffixes = c('.outer1', '.map'))
outer_3 = merge(outer_2, cassini, c('INSEE.ID', 'CLEAN_NAME'), all = TRUE, suffixes = c('.outer2', '.cassini'))

#fix check values
outer_3['INSEE_CHECK'][which(is.na(outer_3['INSEE_CHECK'])),] = FALSE
outer_3['OGEE_CHECK'][which(is.na(outer_3['OGEE_CHECK'])),] = FALSE
outer_3['MAP_CHECK'][which(is.na(outer_3['MAP_CHECK'])),] = FALSE
outer_3['CASSINI_CHECK'][which(is.na(outer_3['CASSINI_CHECK'])),] = FALSE

### Generate a venn diagram (TODO: Other plots)

In [ ]:
m <- as.matrix(data.frame(C1=outer_3['INSEE_CHECK'],C2=outer_3['MAP_CHECK'],
                          C3=outer_3['CASSINI_CHECK'],C4=outer_3['OGEE_CHECK']),
               labels = c('A', 'B', 'C', 'D'))
v = venneuler(m)
v$labels = c('cassini', 'ogee', 'insee', 'map')
plot(v)

In [ ]:
conly = outer_3[which(outer_3['OGEE_CHECK'] == FALSE & outer_3['INSEE_CHECK'] == FALSE & outer_3['MAP_CHECK'] == FALSE),]
monly = outer_3[which(outer_3['OGEE_CHECK'] == FALSE & outer_3['INSEE_CHECK'] == FALSE & outer_3['CASSINI_CHECK'] == FALSE),]
ionly = outer_3[which(outer_3['OGEE_CHECK'] == FALSE & outer_3['MAP_CHECK'] == FALSE & outer_3['CASSINI_CHECK'] == FALSE),]
oonly = outer_3[which(outer_3['CASSINI_CHECK'] == FALSE & outer_3['INSEE_CHECK'] == FALSE & outer_3['MAP_CHECK'] == FALSE),]

write.csv(conly, 'cassini_only.csv')
write.csv(monly, 'map_only.csv')
write.csv(ionly, 'insee_only.csv')
write.csv(oonly, 'ogee_only.csv')

# Section 2: Road Questions

### NOTE: The "BN Ge DD 5548 (3) Saint Brieuc" sheet currently lacks normalized place name, so the processing is currently done on map.csv instead. Uncomment the code in the first box and remove the read.csv to swap to the google sheet. In addition, you will have to remove 'INSEE.ID' from the conditions for the first merge as the sheet does not contain IDs, unlike map.csv. 

## Q1: What is the average distance of a village from a major road? 
### TODO Q2: Are there particular concentrations of villages with road construction (corvée) assignments that are further than 7 kilometers from a road, or are these randomly distributed across the province? Potentially averaging assignment distances by department, or other clusters TBD.
## Q3: Is there any correlation between distance from the city of Rennes and the length of villages’ road construction assignments?
## Q4: How do population proxies (the “communiants” metric from the Ogée data) relate to road construction assignment lengths, and is there regional variation in this? E.g. if population is very small and assignment length is very large (or the opposite), are there patterns for where this happens.

In [ ]:
# url2 = 'https://docs.google.com/spreadsheets/d/1kt5GRQPTy2FBSmo27kZkxclzMNZ3T3MFbpRtkuc2cQ4'

# ## identify this folder on Drive
# ## may prompt you to authorize
# ss2 = drive_get(as_id(url2))

# ## may prompt you to authorize
# map2 = as.data.frame(read_sheet(ss2, sheet = "BN Ge DD 5548 (3) Saint Brieuc"))
# names(map2)[which(names(map2) == 'normalized place name')] = 'normalized.place.name' #fix column name

map2 = read.csv("map.csv")


In [ ]:
#replace accented characters
rep = 'aeecaeiouueiuAEECAEIOUUEIU'
orig = 'àèéçâêîôûùëïüÀÈÉÇÂÊÎÔÛÙËÏÜ'

map2[] <- lapply(map2, chartr, old=orig, new=rep)
map2[] <- lapply(map2, gsub, pattern='Œ', replacement='OE')
map2[] <- lapply(map2, gsub, pattern='œ', replacement='oe')

ogee2[] <- lapply(ogee2, chartr, old=orig, new=rep)
ogee2[] <- lapply(ogee2, gsub, pattern='Œ', replacement='OE')
ogee2[] <- lapply(ogee2, gsub, pattern='œ', replacement='oe')

#add CLEAN_NAME to match with OGEE
nom <- map2$normalized.place.name
nom <- toupper(nom)
map2$CLEAN_NAME <- nom

#regenerate ogee with needed data
nom <- ogee2$'Ogée 1778 entry'
nom <- toupper(nom)
ogee2$CLEAN_NAME <- nom
ogee2[which(is.na(ogee2[,'CLEAN_NAME'])), 'CLEAN_NAME'] = ogee2[which(is.na(ogee2[,'CLEAN_NAME'])), 'NOM']

#enable for a smaller merge, disable to keep all extra metrics
# ogee2_keeps <- c('INSEE.ID', 'CLEAN_NAME', 'distance from Rennes (lieues)', 'communiants')
# ogee2 <- ogee2[ogee2_keeps]

#left merge, preserving all rows from map
left_1 = merge(map2, ogee2, by = c('CLEAN_NAME', 'INSEE.ID'), all.x = TRUE)
left_2 = merge(left_1, insee, by = c('INSEE.ID', 'CLEAN_NAME'), all.x = TRUE)

#remove excess columns
drops = c('INSEE_CHECK', 'OGEE_CHECK')
left_2 <- left_2[ , !(names(left_2) %in% drops)]

In [ ]:
#import shapefile, transform to WGS84
road_dat = shapefile("rennes_corvee_true.shp")
road_dat = spTransform(road_dat, CRS("+proj=longlat +datum=WGS84"))

In [ ]:
#storage for distance from road, decimal latitude, decimal longitude, point on road matched to
road_dist = double()
latdec = numeric()
longdec = numeric()
closept = character()


for(i in 1:nrow(left_2)){
    if(is.na(left_2$long[i]) | is.na(left_2$lat[i])){
        road_dist[i] = NA
        latdec[i] = NA
        longdec[i] =- NA
        next
    }
    lat = left_2$lat[i] #get lat and long
    long = left_2$long[i]
    
    #convert to format for conv_unit
    lat = gsub(' ', '', lat)
    long = gsub(' ', '', long)
    
    lat = gsub('d', ' ', lat)
    long = gsub('d', ' ', long)
    
    lat = gsub('"', '', lat)
    long = gsub('"', '', long)
    
    lat = gsub("'", '.', lat)
    long = gsub("'", '.', long)
    
    lat = gsub("N", "", lat)
    long = gsub("W", "", long)
    
    #convert from DMS to decimal
    lat = as.numeric(conv_unit(lat, from = 'deg_dec_min', to = 'dec_deg'))
    long = as.numeric(paste('-', conv_unit(long, from = 'deg_dec_min', to = 'dec_deg'), sep = ''))
    
    #make point, find nearest point on any road
    pt = matrix(c(long, lat),nrow=1,ncol=2)
    pt = SpatialPoints(pt)
    proj4string(pt) = proj4string(road_dat)
    pts = gNearestPoints(pt, road_dat)
    
    #get matched point on road
    if(lat == ymax(pts)){
        closelat = ymin(pts)
    } else{
        closelat = ymax(pts)
    }
    if(long == xmax(pts)){
        closelong = xmin(pts)
    } else{
        closelong = xmax(pts)
    }
    
    #store resulting points and distance in meters using Vincenty ellipsoid method
    closept[i] = paste(closelat, closelong, sep = ',')
    road_dist[i] = distVincentyEllipsoid(c(xmin(pts), ymin(pts)), c(xmax(pts), ymax(pts)))
    latdec[i] = lat
    longdec[i] = long
}

In [ ]:
#there is a way to calculate which line segment (of 21) is closest, or instead calculate distance to a specific segment
#unfortunately, I've lost that code. It should be possible with the data structures already provided. 

In [ ]:
# save all metrics to csv
left_2['closest_road_latlong'] = closept
left_2['road_dist_m'] = road_dist
left_2['lat_dec'] = latdec
left_2['long_dec'] = longdec

write.csv(left_2, 'map_ogee_insee_left.csv')

## Q1 - mean distance of village from major road (meters)

In [ ]:
mean(road_dist)

## Q3 - Is there any correlation between distance from the city of Rennes and the length of villages’ road construction assignments?

In [ ]:
#note - values will not exist where locations were not matched and no distance was generated
#also will generate incorrect results for malformatted distances from Rennes, as it just converts
#the first number found
assignments = as.numeric(left_2$assignment.distance..toises.)
distances = as.numeric(gsub("([0-9]+).*$", "\\1", left_2[['distance from Rennes (lieues)']]))
plot(distances, assignments, xlab = "Distance from Rennes (lieues)", ylab = "Assignment Distance (toises)")

## Q4 - Population proxies vs assignment lengths

In [ ]:
#note - values will not exist where locations were not matched and no distance was generated.
#current plot bounds are to exclude an outlier with 260000 communiants®
communiants = left_2$communiants
plot(communiants, assignments, xlab = "Communiants", ylab = "Assignment Distance (toises)", 
     xlim=c(0,5000), ylim=c(0,8000))

In [ ]:
# plot the shapefile to make sure it works
# ggplot(road_dat, aes(x=long, y=lat, group=group),) + 
#   geom_path() +
#   theme_classic()